In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vikas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vikas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vikas\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
import random

#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense
#from tensorflow.keras.layers import Dropout

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))
        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [5]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [6]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [7]:
random.shuffle(training)

# Extract X_train (features) and y_train (labels) from training
train_x = np.array([sample[0] for sample in training])  # Features
train_y = np.array([sample[1] for sample in training])
print("Training data created")

Training data created


In [8]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

C:\Users\vikas\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
from tensorflow.keras.optimizers import SGD

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("Model created")

Epoch 1/200
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0263 - loss: 4.1590
Epoch 2/200
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 936us/step - accuracy: 0.0786 - loss: 4.0047
Epoch 3/200
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1115 - loss: 3.8249
Epoch 4/200
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1618 - loss: 3.5419  
Epoch 5/200
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 969us/step - accuracy: 0.2392 - loss: 3.1811
Epoch 6/200
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2557 - loss: 3.0237
Epoch 7/200
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3274 - loss: 2.7392
Epoch 8/200
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3748 - loss: 2.4740
Epoch 9/200
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4096 - loss: 2.2588
Epoch 10/200
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4431 - loss: 2.1373
Epoch 11/200
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4556 - loss: 2.0292
Epoch 12/200
162/162 ━━━━━━━━━━━━━━

Model created
